In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# Python version: 3.6


import os
import copy
import time
import pickle
import numpy as np
from tqdm import tqdm

import torch
from tensorboardX import SummaryWriter

import argparse
from options import args_parser
from update import LocalUpdate, test_inference
from models import MLP, CNNMnist, CNNFashion_Mnist, CNNCifar
from utils import get_dataset, average_weights, exp_details




In [2]:
# if __name__ == '__main__':
start_time = time.time()

# define paths
path_project = os.path.abspath('..')
logger = SummaryWriter('../logs')
print('here3')

args = args_parser()
print('here2')
exp_details(args)
print('here1')
device = 'cuda' if args.gpu else 'cpu'

if args.gpu:
    torch.cuda.set_device(args.gpu)

    
    
# load dataset and user groups
train_dataset, test_dataset, user_groups = get_dataset(args)

# BUILD MODEL
if args.model == 'cnn':
    # Convolutional neural netork
    if args.dataset == 'mnist':
        global_model = CNNMnist(args=args)
    elif args.dataset == 'fmnist':
        global_model = CNNFashion_Mnist(args=args)
    elif args.dataset == 'cifar':
        global_model = CNNCifar(args=args)

elif args.model == 'mlp':
    # Multi-layer preceptron
    img_size = train_dataset[0][0].shape
    len_in = 1
    for x in img_size:
        len_in *= x
        global_model = MLP(dim_in=len_in, dim_hidden=64,
                           dim_out=args.num_classes)
else:
    print('Error: unrecognized model')



here3
here2

Experimental details:
    Model     : mlp
    Optimizer : sgd
    Learning  : 0.01
    Global Rounds   : 10

    Federated parameters:
    IID
    Fraction of users  : 0.1
    Local Batch size   : 10
    Local Epochs       : 10

here1


In [3]:
from matplotlib import pyplot as plt


In [4]:

# num_users=100
# test=np.random.exponential(scale=1.0, size=num_users)
# print(test)
# plt.hist(test)
# plt.show()
# norm=np.random.normal(loc=1.19865, scale=1.0, size=num_users)
# print(norm)
# plt.plot(np.arange(norm.size),norm)
# plt.show()

In [5]:
# (-0.4242+2.8215)/2

In [6]:
# def gen_response_time(num_users=args.num_users):
#     np.random.exponential(scale=1.0, size=num_users)
    

In [7]:
# Set the model to train and send it to device.
global_model.to(device)
global_model.train()
print(global_model)

# copy weights
global_weights = global_model.state_dict()

# Training
train_loss, train_accuracy = [], []
val_acc_list, net_list = [], []
cv_loss, cv_acc = [], []
print_every = 2
val_loss_pre, counter = 0, 0
res_times_epochs = []
selected_user_idxs_epochs=[]
average_time_epochs=[]
test_accs_epochs=[]
noise_levels_epochs=[]
seed=20
max_time_epochs=[]
# np.random.seed(seed)
R = 10
threshold=0.7

for epoch in tqdm(range(args.epochs)):
    test_accs=np.zeros(args.num_users)
    local_weights=np.zeros(args.num_users)
    local_losses = np.zeros(args.num_users)
    print(f'\n | Global Training Round : {epoch+1} |\n')
    res_times=np.random.exponential(scale=10, size=args.num_users)
    res_times_epochs.append(res_times)
    print(res_times[:10])
    sorted_time_idx=np.argsort(res_times, axis=- 1, kind=None, order=None)
    noise_levels = np.random.uniform(low=0, high=1, size=args.num_users) # noise level for norm dist, 0 mean, this is var
    noise_levels_epochs.append(noise_levels)
    global_model.train()
    # m = max(int(args.frac * args.num_users), 1)
    # idxs_users = np.random.choice(range(args.num_users), m, replace=False)
    idxs_users = np.arange(args.num_users)
    # selected_user_idxs_epochs.append(idxs_users)
    for idx in sorted_time_idx:
        local_model = LocalUpdate(args=args, dataset=train_dataset,
                                  idxs=user_groups[idx], logger=logger, noise_level=noise_levels[idx], seed=seed)
        w, loss, test_acc = local_model.update_weights(
            model=copy.deepcopy(global_model), global_round=epoch)
        local_weights[idx]=(copy.deepcopy(w))
        local_losses[idx]=(copy.deepcopy(loss))
        test_accs[idx]=test_acc
        
        selected_user_idxs=[]
        selected_local_weights=[]

        average_time=0
        max_time=0
        
        if len(selected_user_idxs) + (args.num_users-idx) <= R:
            selected_user_idxs.append(sorted_time_idx[idx])
            selected_local_weights.append(copy.deepcopy(local_weights[sorted_time_idx[idx]]))
            average_time+=res_times[sorted_time_idx[idx]]
            max_time=max(max_time, res_times[sorted_time_idx[idx]])

        
        elif test_accs[sorted_time_idx[idx]] > threshold:        
            selected_user_idxs.append(sorted_time_idx[idx])
            selected_local_weights.append(copy.deepcopy(local_weights[sorted_time_idx[idx]]))
            average_time+=res_times[sorted_time_idx[idx]]
            max_time=max(max_time, res_times[sorted_time_idx[idx]])
            
        if len(selected_user_idxs)==R:
            break;
        
    test_accs_epochs.append(test_accs)
    

#     for i in range(args.num_users):
#         if len(selected_user_idxs) + (args.num_users-i) <= R:
#             selected_user_idxs.append(sorted_time_idx[i])
#             selected_local_weights.append(copy.deepcopy(local_weights[sorted_time_idx[i]]))
#             average_time+=res_times[sorted_time_idx[i]]
#             max_time=max(max_time, res_times[sorted_time_idx[i]])

        
#         elif test_accs[sorted_time_idx[i]] > threshold:        
#             selected_user_idxs.append(sorted_time_idx[i])
#             selected_local_weights.append(copy.deepcopy(local_weights[sorted_time_idx[i]]))
#             average_time+=res_times[sorted_time_idx[i]]
#             max_time=max(max_time, res_times[sorted_time_idx[i]])
            
#         if len(selected_user_idxs)==R:
#             break;
#         if test_accs
#         average_time+=res_times[idx]
#         max_time = max(max_time, res_times[idx])
    
    average_time = average_time/len(idxs_users)
    average_time_epochs.append(average_time)
    max_time_epochs.append(max_time)
    selected_user_idxs_epochs.append(selected_user_idxs)
    # update global weights
    # global_weights = average_weights(local_weights)
    global_weights = average_weights(selected_local_weights)
    
    # update global weights
    global_model.load_state_dict(global_weights)

    loss_avg = sum(local_losses) / len(local_losses)
    train_loss.append(loss_avg)

    # Calculate avg training accuracy over all users at every epoch
    list_acc, list_loss = [], []
    global_model.eval()
    for c in range(args.num_users):
        local_model = LocalUpdate(args=args, dataset=train_dataset,
                                  idxs=user_groups[idx], logger=logger, noise_level=noise_levels[idx], seed=seed)
        acc, loss = local_model.inference(model=global_model)
        list_acc.append(acc)
        list_loss.append(loss)
    train_accuracy.append(sum(list_acc)/len(list_acc))

    # print global training loss after every 'i' rounds
    if (epoch+1) % print_every == 0:
        print(f' \nAvg Training Stats after {epoch+1} global rounds:')
        print(f'Training Loss : {np.mean(np.array(train_loss))}')
        print('Train Accuracy: {:.2f}% \n'.format(100*train_accuracy[-1]))



MLP(
  (layer_input): Linear(in_features=784, out_features=64, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (layer_hidden): Linear(in_features=64, out_features=10, bias=True)
  (softmax): Softmax(dim=1)
)


  0%|                                                    | 0/10 [00:00<?, ?it/s]


 | Global Training Round : 1 |

[ 4.99249047 17.21465396 21.39139132  2.19515846 11.03112953  6.39964461
  1.6486552   8.07050254  0.50586462 14.99218015]


/Users/xin/Developer/CMPUT675_Project/src/update.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(image), torch.tensor(label)
  0%|                                                    | 0/10 [00:01<?, ?it/s]

test_accu: 0.5666666666666667  test loss: -4.088653311133385


TypeError: float() argument must be a string or a number, not 'collections.OrderedDict'

In [ ]:
# Test inference after completion of training
test_acc, test_loss = test_inference(args, global_model, test_dataset)

print(f' \n Results after {args.epochs} global rounds of training:')
print("|---- Avg Train Accuracy: {:.2f}%".format(100*train_accuracy[-1]))
print("|---- Test Accuracy: {:.2f}%".format(100*test_acc))

# Saving the objects train_loss and train_accuracy:
file_name = '../save/objects/{}_{}_{}_C[{}]_iid[{}]_E[{}]_B[{}].pkl'.\
    format(args.dataset, args.model, args.epochs, args.frac, args.iid,
           args.local_ep, args.local_bs)

with open(file_name, 'wb') as f:
    pickle.dump([train_loss, train_accuracy], f)

print('\n Total Run Time: {0:0.4f}'.format(time.time()-start_time))

print(np.mean(average_time_epochs))
print(np.mean(max_time_epochs))

In [ ]:
print(average_time_epochs)
print(max_time_epochs)

In [ ]:
# PLOTTING (optional)
# import matplotlib
# import matplotlib.pyplot as plt
# matplotlib.use('Agg')

# Plot Loss curve
# plt.figure()
# plt.title('Training Loss vs Communication rounds')
# plt.plot(range(len(train_loss)), train_loss, color='r')
# plt.ylabel('Training loss')
# plt.xlabel('Communication Rounds')
# plt.savefig('../save/fed_{}_{}_{}_C[{}]_iid[{}]_E[{}]_B[{}]_loss.png'.
#             format(args.dataset, args.model, args.epochs, args.frac,
#                    args.iid, args.local_ep, args.local_bs))
#
# # Plot Average Accuracy vs Communication rounds
# plt.figure()
# plt.title('Average Accuracy vs Communication rounds')
# plt.plot(range(len(train_accuracy)), train_accuracy, color='k')
# plt.ylabel('Average Accuracy')
# plt.xlabel('Communication Rounds')
# plt.savefig('../save/fed_{}_{}_{}_C[{}]_iid[{}]_E[{}]_B[{}]_acc.png'.
#             format(args.dataset, args.model, args.epochs, args.frac,
#                    args.iid, args.local_ep, args.local_bs))


In [ ]:
(79.39+73.32+70.74+76.65+77.48)/5

In [ ]:
(100/30+15*15/60+800/60)/5*72

In [ ]:
214.0625/15/(2*(1-13.75/15))

In [ ]:
0.5 * 95.625 + 0.25 * 100.625 + 0.25 *105.625

In [ ]:
0.5  *  25.3125 + 0.25  *51.75 + 0.25  *314

In [ ]:
(100/30+15*15/60)/5*9

In [ ]:
13.75*13.75/15/(2*(1-13.75/15))

In [ ]:
0.5 * 15.3125 + 0.25 *27.75 + 0.25 *342

In [ ]:
0.5*10+0.25*15+ 0.25*20